In [55]:
import pandas as pd
import pickle
import numpy as np

a_cols = pickle.load(open('../data/mappings/Accolades_columns.pkl' ,'rb'))
c_cols = pickle.load(open('../data/mappings/Census_columns.pkl' ,'rb'))
cs_dict = pickle.load(open('../data/mappings/CollegeScorecard_columns.pkl' ,'rb'))
cs_cols = [col for cols in cs_dict.values() for col in cols]
sp_dict = pickle.load(open('../data/mappings/SportParticipation_columns.pkl' ,'rb'))
sp_cols = [col for cols in sp_dict.values() for col in cols]

df = pd.read_pickle('../data/merged/full_df.pkl')
original_df = df.copy()
df[sp_cols] = df[sp_cols].fillna(0)
df[a_cols] = df[a_cols].fillna(0)
df = df.replace({pd.NA: np.nan})
df

,Year,UNITID,Institution Name,LONGITUDE,LATITUDE,City,State Code,Region Name,School Type,Predominant Degree,...,Women's Lacrosse Champion,Women's Outdoor Track & Field Champion,Women's Soccer Champion,Women's Softball Champion,Women's Swimming & Diving Champion,Women's Tennis Champion,Women's Volleyball Champion,Total Champions,State Population,US Population
0,2004,100654,Alabama A & M University,-86.568502,34.783368,Normal,AL,Southeast,Public,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4530729.0,292805298.0
1,2004,100663,University of Alabama at Birmingham,-86.799345,33.505697,Birmingham,AL,Southeast,Public,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4530729.0,292805298.0
2,2004,100706,University of Alabama in Huntsville,-86.640449,34.724557,Huntsville,AL,Southeast,Public,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4530729.0,292805298.0
3,2004,100724,Alabama State University,-86.295677,32.364317,Montgomery,AL,Southeast,Public,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4530729.0,292805298.0
4,2004,100751,The University of Alabama,-87.545978,33.211875,Tuscaloosa,AL,Southeast,Public,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4530729.0,292805298.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20281,2023,486840,Kennesaw State University,-84.583052,34.038807,Kennesaw,GA,Southeast,Public,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11064432.0,336806231.0
20282,2023,487524,Husson University,-68.791476,44.829002,Bangor,ME,New England,Private Nonprofit,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1399646.0,336806231.0
20283,2023,490805,Purdue University Northwest,-87.474236,41.584324,Hammond,IN,Great Lakes,Public,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6880131.0,336806231.0
20284,2023,498562,Commonwealth University of Pennsylvania,-76.447844,41.007820,Bloomsburg,PA,Mid East,Public,<NA>,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13017721.0,336806231.0


In [56]:
demographic_cols = cs_dict['numerical'] + cs_dict['categorical']
demographic_cols = [
    col for col in demographic_cols
    if col not in ['Sanction Name', 'Division Name', 'Football Subdivision']
]

missing_flag_cols = []
for col in demographic_cols:
    if col in df.columns:
        flag_col = f"{col} Missing"
        df[flag_col] = df[col].isna().astype(int)
        missing_flag_cols.append(flag_col)

numeric_demo = [
    c for c in demographic_cols
    if c in df.columns and pd.api.types.is_numeric_dtype(df[c])
]
categorical_demo = [
    c for c in demographic_cols
    if c in df.columns and not pd.api.types.is_numeric_dtype(df[c])
]

for col in numeric_demo:
    s = df[col]
    df[col] = s.astype("float64")
    df[col] = df[col].fillna(df[col].median())

for col in categorical_demo:
    df[col] = df[col].astype("string")
    df[col] = df[col].fillna("Missing")

demographic_cols

['Median ACT English',
 'Median ACT Math',
 'Median ACT Writing',
 'Median ACT Score',
 'Average SAT Score',
 'In-State Tuition',
 'Out-of-State Tuition',
 'Estimated Cost of Attendence',
 'Admissions Rate',
 'Average Faculty Salary',
 'Median Earnings after 10yrs',
 'Student to Faculty Ratio',
 'Off-Campus Room and Board',
 'Male Undergraduates',
 'Female Undergraduates',
 'Total Undergraduates',
 'Total Grad Students',
 'Total Enrollment',
 'City',
 'State Code',
 'Region Name',
 'School Type',
 'Predominant Degree',
 'Highest Degree',
 'HBCU',
 'Hispanic Serving College']

In [57]:
champ_to_prefix = {
    "Men's Baseball Champion": "Baseball Men's Team",
    "Men's Basketball Champion": "Basketball Men's Team",
    "Men's Cross Country Champion": "Track and Field X Country Men's Team",
    "Men's Football Champion": "Football Men's Team",
    "Men's Golf Champion": "Golf Men's Team",
    "Men's Gymnastics Champion": "Gymnastics Men's Team",
    "Men's Indoor Track & Field Champion": "Track and Field Indoor Men's Team",
    "Men's Lacrosse Champion": "Lacrosse Men's Team",
    "Men's Outdoor Track & Field Champion": "Track and Field Outdoor Men's Team",
    "Men's Soccer Champion": "Soccer Men's Team",
    "Men's Swimming & Diving Champion": "Swimming and Diving Men's Team",
    "Men's Tennis Champion": "Tennis Men's Team",
    "Men's Volleyball Champion": "Volleyball Men's Team",
    "Men's Wrestling Champion": "Wrestling Men's Team",

    "Women's Basketball Champion": "Basketball Women's Team",
    "Women's Cross Country Champion": "Track and Field X Country Women's Team",
    "Women's Golf Champion": "Golf Women's Team",
    "Women's Gymnastics Champion": "Gymnastics Women's Team",
    "Women's Indoor Track & Field Champion": "Track and Field Indoor Women's Team",
    "Women's Lacrosse Champion": "Lacrosse Women's Team",
    "Women's Outdoor Track & Field Champion": "Track and Field Outdoor Women's Team",
    "Women's Soccer Champion": "Soccer Women's Team",
    "Women's Softball Champion": "Softball Women's Team",
    "Women's Swimming & Diving Champion": "Swimming and Diving Women's Team",
    "Women's Tennis Champion": "Tennis Women's Team",
    "Women's Volleyball Champion": "Volleyball Women's Team",
}

def metric_columns_for_prefix(prefix: str, cols, metrics=("Participation", "Revenue", "Expenses")):
    out = []
    col_set = set(cols)
    for metric in metrics:
        col_name = f"{prefix} {metric}"
        if col_name in col_set:
            out.append(col_name)
    return out

def build_champ_to_all_metrics(cols):
    champ_to_metrics = {}

    for champ, prefix in champ_to_prefix.items():
        metrics_cols = metric_columns_for_prefix(
            prefix,
            cols,
            metrics=("Participation", "Revenue", "Expenses"),
        )
        champ_to_metrics[champ] = metrics_cols

    total_metric_cols = sorted({
        col
        for champ, col_list in champ_to_metrics.items()
        for col in col_list
    })

    champ_to_metrics["Total Champions"] = total_metric_cols
    return champ_to_metrics

cols = list(df.columns)
champ_to_all_metrics = build_champ_to_all_metrics(cols)
champ_to_all_metrics

{"Men's Baseball Champion": ["Baseball Men's Team Participation",
  "Baseball Men's Team Revenue",
  "Baseball Men's Team Expenses"],
 "Men's Basketball Champion": ["Basketball Men's Team Participation",
  "Basketball Men's Team Revenue",
  "Basketball Men's Team Expenses"],
 "Men's Cross Country Champion": ["Track and Field X Country Men's Team Participation",
  "Track and Field X Country Men's Team Revenue",
  "Track and Field X Country Men's Team Expenses"],
 "Men's Football Champion": ["Football Men's Team Participation",
  "Football Men's Team Revenue",
  "Football Men's Team Expenses"],
 "Men's Golf Champion": ["Golf Men's Team Participation",
  "Golf Men's Team Revenue",
  "Golf Men's Team Expenses"],
 "Men's Gymnastics Champion": ["Gymnastics Men's Team Participation",
  "Gymnastics Men's Team Revenue",
  "Gymnastics Men's Team Expenses"],
 "Men's Indoor Track & Field Champion": ["Track and Field Indoor Men's Team Participation",
  "Track and Field Indoor Men's Team Revenue",
 

In [58]:
import numpy as np
import pandas as pd

df = df.copy()

if "Football Subdivision" in df.columns:
    df["Football Subdivision Clean"] = df["Football Subdivision"].fillna("")
else:
    df["Football Subdivision Clean"] = ""

group_cols_nonfb = ["Year", "Sanction Name", "Division Name"]
group_cols_fb = ["Year", "Sanction Name", "Division Name", "Football Subdivision Clean"]

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

df = df.copy()
if "Football Subdivision" in df.columns:
    df["Football Subdivision Clean"] = df["Football Subdivision"].fillna("")
else:
    df["Football Subdivision Clean"] = ""

group_cols_nonfb = ["Year", "Sanction Name", "Division Name"]
group_cols_fb = ["Year", "Sanction Name", "Division Name", "Football Subdivision Clean"]

id_cols = [c for c in ["UNITID", "Institution Name", "School Name", "INSTNM"] if c in df.columns]

group_cols_all = list(set(group_cols_nonfb + group_cols_fb + ["Football Subdivision"]))
exclude_as_features = set(group_cols_all)

results_by_champ = {}
winner_rows = []
all_rows = []

for champ_name, metric_cols in champ_to_all_metrics.items():
    if champ_name == "Total Champions":
        continue

    if champ_name not in df.columns:
        print(f"Skipping {champ_name}: target column not in df.")
        continue

    sport_metric_cols = champ_to_all_metrics[champ_name]
    participation_col = next((c for c in sport_metric_cols if "Participation" in c), None)
    revenue_col = next((c for c in sport_metric_cols if "Revenue" in c), None)
    expenses_col = next((c for c in sport_metric_cols if "Expenses" in c), None)

    metric_cols_present = [c for c in metric_cols if c in df.columns]
    demo_cols_present = [
        c for c in demographic_cols
        if c in df.columns and c not in exclude_as_features
    ]
    feature_cols = metric_cols_present + demo_cols_present

    if not feature_cols:
        print(f"Skipping {champ_name}: no feature columns present.")
        continue

    cols_needed = (
            feature_cols
            + group_cols_nonfb
            + ["Football Subdivision Clean", champ_name]
            + id_cols
    )

    tmp = df[cols_needed].dropna().reset_index(drop=True)
    if tmp.empty:
        print(f"Skipping {champ_name}: no non-missing data.")
        continue

    if tmp[champ_name].nunique() < 2:
        print(f"Skipping {champ_name}: target has <2 classes in all data.")
        continue

    X_all = tmp[feature_cols]
    y_all = tmp[champ_name].astype(int)

    numeric_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df[c])]
    categorical_cols = [c for c in feature_cols if c not in numeric_cols]

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", "passthrough", numeric_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ],
        remainder="drop",
    )

    model = Pipeline([
        ("prep", preprocessor),
        ("clf", RandomForestClassifier(
            n_estimators=200,
            max_depth=None,
            random_state=42,
            class_weight="balanced",
            n_jobs=-1,
        )),
    ])

    model.fit(X_all, y_all)
    probs_all = model.predict_proba(X_all)[:, 1]

    base_cols = group_cols_nonfb + ["Football Subdivision Clean"] + id_cols
    pred_df = tmp[base_cols].copy()
    pred_df["y_true"] = y_all.values
    pred_df["y_prob"] = probs_all

    if champ_name == "Men's Football Champion":
        group_cols_this = group_cols_fb
    else:
        group_cols_this = group_cols_nonfb

    y_pred_one = np.zeros(len(pred_df), dtype=int)
    group_results = {}

    for group_vals, idxs in pred_df.groupby(group_cols_this).groups.items():
        idxs = np.array(list(idxs))
        if len(idxs) == 0:
            continue

        y_true_group = pred_df.loc[idxs, "y_true"].values
        actual_idxs = idxs[y_true_group == 1]

        if len(actual_idxs) == 0:
            continue

        for idx in idxs:
            row = pred_df.loc[idx]

            year_val = row["Year"]
            sanc_val = row["Sanction Name"]
            div_val = row["Division Name"]
            fb_sub_val = (
                row["Football Subdivision Clean"]
                if champ_name == "Men's Football Champion" else ""
            )

            row_dict_all = {
                "Champion": champ_name,
                "Year": year_val,
                "Sanction Name": sanc_val,
                "Division Name": div_val,
                "Football Subdivision Clean": fb_sub_val,
                "Predicted Prob": float(row["y_prob"]),
                "True Champion": bool(row["y_true"] == 1),
                "Predicted Champion": False,
            }

            for col in id_cols:
                row_dict_all[col] = row[col]

            row_dict_all["Participation"] = (
                tmp.loc[idx, participation_col] if participation_col else None
            )
            row_dict_all["Revenue"] = (
                tmp.loc[idx, revenue_col] if revenue_col else None
            )
            row_dict_all["Expenses"] = (
                tmp.loc[idx, expenses_col] if expenses_col else None
            )

            row_dict_all["_local_idx"] = idx

            all_rows.append(row_dict_all)

        if participation_col is not None and participation_col in tmp.columns:
            part_vals = tmp.loc[idxs, participation_col].values.astype(float)
            valid_idxs = idxs[part_vals > 0]
        else:
            valid_idxs = idxs

        if len(valid_idxs) == 0:
            acc_group = np.nan
            champion_correct = False
        else:
            group_probs = pred_df.loc[valid_idxs, "y_prob"].values
            best_local = valid_idxs[np.argmax(group_probs)]
            y_pred_one[best_local] = 1

            y_pred_group = (y_pred_one[idxs] == 1).astype(int)

            acc_group = accuracy_score(y_true_group, y_pred_group)
            true_pos_idx = np.where(y_true_group == 1)[0]
            predicted_pos_idx = np.where(y_pred_group == 1)[0]
            champion_correct = (
                    len(true_pos_idx) == 1
                    and len(predicted_pos_idx) == 1
                    and true_pos_idx[0] == predicted_pos_idx[0]
            )

            winner_row = pred_df.loc[best_local]

            year_val = winner_row["Year"]
            sanc_val = winner_row["Sanction Name"]
            div_val = winner_row["Division Name"]
            fb_sub_val = (
                winner_row["Football Subdivision Clean"]
                if champ_name == "Men's Football Champion"
                else ""
            )

            row_dict_pred = {
                "Champion": champ_name,
                "Year": year_val,
                "Sanction Name": sanc_val,
                "Division Name": div_val,
                "Football Subdivision Clean": fb_sub_val,
                "Predicted Prob": winner_row["y_prob"],
                "True Champion": bool(winner_row["y_true"] == 1),
                "Predicted Champion": True,
            }

            for col in id_cols:
                row_dict_pred[col] = winner_row[col]

            row_dict_pred["Participation"] = (
                tmp.loc[best_local, participation_col] if participation_col else None
            )
            row_dict_pred["Revenue"] = (
                tmp.loc[best_local, revenue_col] if revenue_col else None
            )
            row_dict_pred["Expenses"] = (
                tmp.loc[best_local, expenses_col] if expenses_col else None
            )

            winner_rows.append(row_dict_pred)

            for r in all_rows:
                if (
                        r["Champion"] == champ_name
                        and r["Year"] == year_val
                        and r["Sanction Name"] == sanc_val
                        and r["Division Name"] == div_val
                        and r.get("_local_idx", None) == best_local
                ):
                    r["Predicted Champion"] = True

        group_results[group_vals] = {
            "n_rows": len(idxs),
            "accuracy_one_winner": acc_group,
            "champion_correct": champion_correct,
        }

        for actual_idx in actual_idxs:
            actual_row = pred_df.loc[actual_idx]

            row_dict_actual = {
                "Champion": champ_name,
                "Year": actual_row["Year"],
                "Sanction Name": actual_row["Sanction Name"],
                "Division Name": actual_row["Division Name"],
                "Football Subdivision Clean": (
                    actual_row["Football Subdivision Clean"]
                    if champ_name == "Men's Football Champion" else ""
                ),
                "Predicted Prob": pred_df.loc[actual_idx, "y_prob"],
                "True Champion": True,
                "Predicted Champion": False,
            }

            for col in id_cols:
                row_dict_actual[col] = actual_row[col]

            row_dict_actual["Participation"] = (
                tmp.loc[actual_idx, participation_col] if participation_col else None
            )
            row_dict_actual["Revenue"] = (
                tmp.loc[actual_idx, revenue_col] if revenue_col else None
            )
            row_dict_actual["Expenses"] = (
                tmp.loc[actual_idx, expenses_col] if expenses_col else None
            )

            winner_rows.append(row_dict_actual)

    overall_acc = accuracy_score(pred_df["y_true"].values, y_pred_one)

    results_by_champ[champ_name] = {
        "model": model,
        "feature_cols": feature_cols,
        "numeric_cols": numeric_cols,
        "categorical_cols": categorical_cols,
        "group_cols_used": group_cols_this,
        "overall_accuracy_one_winner": overall_acc,
        "group_results": group_results,
        "n_samples": len(pred_df),
    }

    print(f"{champ_name}: overall one-winner accuracy = {overall_acc:.3f}, "
          f"groups = {len(group_results)}")

winners_df = pd.DataFrame(winner_rows)

all_df = pd.DataFrame(all_rows)
if "_local_idx" in all_df.columns:
    all_df = all_df.drop(columns=["_local_idx"])

cols_order = [
    "Champion", "Year", "Sanction Name", "Division Name", "Football Subdivision Clean",
    *[c for c in ["UNITID", "Institution Name", "School Name", "INSTNM"] if c in all_df.columns],
    "Participation", "Revenue", "Expenses",
    "Predicted Prob", "True Champion", "Predicted Champion",
]
all_df = all_df[[c for c in cols_order if c in all_df.columns]]
# winners_df = winners_df.iloc[:, :12]

Men's Baseball Champion: overall one-winner accuracy = 1.000, groups = 49
Men's Basketball Champion: overall one-winner accuracy = 1.000, groups = 48
Men's Cross Country Champion: overall one-winner accuracy = 0.997, groups = 38
Men's Football Champion: overall one-winner accuracy = 1.000, groups = 72
Men's Golf Champion: overall one-winner accuracy = 1.000, groups = 49
Men's Gymnastics Champion: overall one-winner accuracy = 1.000, groups = 19
Men's Indoor Track & Field Champion: overall one-winner accuracy = 0.997, groups = 42
Men's Lacrosse Champion: overall one-winner accuracy = 1.000, groups = 2
Men's Outdoor Track & Field Champion: overall one-winner accuracy = 0.996, groups = 45
Men's Soccer Champion: overall one-winner accuracy = 1.000, groups = 48
Men's Swimming & Diving Champion: overall one-winner accuracy = 1.000, groups = 48
Men's Tennis Champion: overall one-winner accuracy = 1.000, groups = 46
Men's Volleyball Champion: overall one-winner accuracy = 0.999, groups = 20
Me

In [68]:
all_df

,Champion,Year,Sanction Name,Division Name,Football Subdivision Clean,UNITID,Institution Name,Participation,Revenue,Expenses,Predicted Prob,True Champion,Predicted Champion
0,Men's Baseball Champion,2004,NCAA,Division 2,,100706,University of Alabama in Huntsville,33.0,254334.0,237479.0,0.000,False,False
1,Men's Baseball Champion,2004,NCAA,Division 2,,101587,University of West Alabama,41.0,155567.0,202500.0,0.000,False,False
2,Men's Baseball Champion,2004,NCAA,Division 2,,101675,Miles College,16.0,38558.0,54381.0,0.000,False,False
3,Men's Baseball Champion,2004,NCAA,Division 2,,101709,University of Montevallo,28.0,247255.0,247255.0,0.005,False,False
4,Men's Baseball Champion,2004,NCAA,Division 2,,101879,University of North Alabama,28.0,280275.0,280275.0,0.000,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355614,Women's Volleyball Champion,2023,NCAA,Division 3,,240462,University of Wisconsin-Platteville,20.0,314870.0,287225.0,0.000,False,False
355615,Women's Volleyball Champion,2023,NCAA,Division 3,,240471,University of Wisconsin-River Falls,18.0,242233.0,242233.0,0.000,False,False
355616,Women's Volleyball Champion,2023,NCAA,Division 3,,240480,University of Wisconsin-Stevens Point,20.0,209125.0,214390.0,0.000,False,False
355617,Women's Volleyball Champion,2023,NCAA,Division 3,,366252,Pennsylvania College of Technology,11.0,36723.0,36723.0,0.000,False,False


In [61]:
winners_df[(winners_df['Champion']=="Men's Lacrosse Champion")]

,Champion,Year,Sanction Name,Division Name,Football Subdivision Clean,Predicted Prob,True Champion,Predicted Champion,UNITID,Institution Name,Participation,Revenue,Expenses
638,Men's Lacrosse Champion,2023,NCAA,Division 2,,0.705,True,True,198835,Lenoir-Rhyne University,58.0,741615.0,741615.0
639,Men's Lacrosse Champion,2023,NCAA,Division 2,,0.705,True,False,198835,Lenoir-Rhyne University,58.0,741615.0,741615.0
640,Men's Lacrosse Champion,2023,NCAA,Division 3,,0.690,True,True,163851,Salisbury University,59.0,343653.0,343653.0
641,Men's Lacrosse Champion,2023,NCAA,Division 3,,0.690,True,False,163851,Salisbury University,59.0,343653.0,343653.0


In [62]:
demographic_df = original_df[['UNITID', 'Year', 'LATITUDE', 'LONGITUDE'] + demographic_cols]
demographic_df

,UNITID,Year,LATITUDE,LONGITUDE,Median ACT English,Median ACT Math,Median ACT Writing,Median ACT Score,Average SAT Score,In-State Tuition,...,Total Grad Students,Total Enrollment,City,State Code,Region Name,School Type,Predominant Degree,Highest Degree,HBCU,Hispanic Serving College
0,100654,2004,34.783368,-86.568502,17.0,17.0,NaN,18.0,850.0,4420,...,1232,5956.0,Normal,AL,Southeast,Public,3,4,1,0
1,100663,2004,33.505697,-86.799345,21.0,21.0,NaN,22.0,1037.0,4662,...,4275,12270.0,Birmingham,AL,Southeast,Public,3,4,0,0
2,100706,2004,34.724557,-86.640449,25.0,24.0,NaN,25.0,1133.0,4516,...,1513,5546.0,Huntsville,AL,Southeast,Public,3,4,0,0
3,100724,2004,32.364317,-86.295677,15.0,16.0,NaN,16.0,783.0,4008,...,964,5109.0,Montgomery,AL,Southeast,Public,3,4,1,0
4,100751,2004,33.211875,-87.545978,25.0,23.0,NaN,24.0,1091.0,4630,...,3754,18609.0,Tuscaloosa,AL,Southeast,Public,3,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20281,486840,2023,34.038807,-84.583052,NaN,NaN,NaN,NaN,1120.0,5786,...,4560,34544.0,Kennesaw,GA,Southeast,Public,3,4,0,0
20282,487524,2023,44.829002,-68.791476,NaN,NaN,NaN,NaN,1110.0,22194,...,608,2590.0,Bangor,ME,New England,Private Nonprofit,3,4,0,0
20283,490805,2023,41.584324,-87.474236,NaN,NaN,NaN,NaN,1034.0,8419,...,777,5399.0,Hammond,IN,Great Lakes,Public,3,4,0,1
20284,498562,2023,41.007820,-76.447844,NaN,NaN,NaN,NaN,1093.0,11046,...,1121,9839.0,Bloomsburg,PA,Mid East,Public,<NA>,4,0,0


In [93]:
merged = pd.merge(all_df, demographic_df, on=['Year', 'UNITID'], how='left')
merged = merged.rename(columns={'Champion': 'Sport Team'})
merged['Champion'] = pd.Series(index=merged.index, dtype='object')

mask_pred = merged['Predicted Champion'].astype(bool)
mask_true = merged['True Champion'].astype(bool)
mask_both = mask_pred & mask_true
merged['Champion'] = 'Not Champion'
merged.loc[mask_pred, 'Champion'] = 'Predicted Champion'
merged.loc[mask_true, 'Champion'] = 'True Champion'
merged.loc[mask_both, 'Champion'] = 'Predicted and True Champion'

merged[merged['True Champion']].columns

Index(['Sport Team', 'Year', 'Sanction Name', 'Division Name',
       'Football Subdivision Clean', 'UNITID', 'Institution Name',
       'Participation', 'Revenue', 'Expenses', 'Predicted Prob',
       'True Champion', 'Predicted Champion', 'LATITUDE', 'LONGITUDE',
       'Median ACT English', 'Median ACT Math', 'Median ACT Writing',
       'Median ACT Score', 'Average SAT Score', 'In-State Tuition',
       'Out-of-State Tuition', 'Estimated Cost of Attendence',
       'Admissions Rate', 'Average Faculty Salary',
       'Median Earnings after 10yrs', 'Student to Faculty Ratio',
       'Off-Campus Room and Board', 'Male Undergraduates',
       'Female Undergraduates', 'Total Undergraduates', 'Total Grad Students',
       'Total Enrollment', 'City', 'State Code', 'Region Name', 'School Type',
       'Predominant Degree', 'Highest Degree', 'HBCU',
       'Hispanic Serving College', 'Champion'],
      dtype='object')

In [91]:
merged.to_csv('../data/tableau/random_forest.csv', index=False)